### Layers : Fundamental blocks of Neural Network

In [1]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable



### Linear(in_features,out_features,bias)

定義 輸入和輸出層的維度(weight的維度)

In [2]:
myLayer = Linear(in_features=10,out_features=5,bias=True)
inp = Variable(torch.randn(1,10))
myLayer = Linear(in_features=10,out_features=5,bias=True) 
myLayer(inp)

tensor([[-0.0849,  0.0905,  0.7727,  1.3033,  0.1889]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[-0.2387,  0.2461,  0.0400,  0.3149, -0.0983, -0.2933, -0.2519,  0.1193,
          0.1335, -0.2589],
        [ 0.1351,  0.0382,  0.2531,  0.1099,  0.1777, -0.2558, -0.1711, -0.0573,
          0.0229, -0.1513],
        [ 0.1416, -0.1679,  0.0619,  0.0597, -0.2159, -0.3015, -0.0551,  0.2016,
         -0.2469, -0.2178],
        [ 0.1052, -0.1095, -0.0971,  0.2633,  0.0479,  0.1643, -0.2891,  0.2123,
         -0.2719, -0.2520],
        [ 0.2962,  0.1138,  0.2381,  0.2380, -0.1595, -0.0852, -0.0812, -0.1172,
         -0.1722, -0.2352]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([ 0.1728,  0.0194,  0.2135, -0.3099,  0.2750], requires_grad=True)

### Stacking Linear layers

建立多層神經層透過迭代

In [5]:
myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

tensor([[-0.0013,  0.7794]], grad_fn=<AddmmBackward>)

### PyTorch Non-linear Activations

使用兩種呼叫激勵函數的不同方式

In [6]:
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
myRelu = ReLU()
myRelu(sample_data)


tensor([[1., 2., 0., 0.]])

In [7]:
import torch.nn as nn
import torch.nn.functional as F
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
f = F.relu(sample_data) # Much simpler.
f

tensor([[1., 2., 0., 0.]])

### Neural Network 

利用類別建立多層神經網路

In [8]:
class MyFirstNetwork(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyFirstNetwork,self).__init__() 
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def layer(self):
        first  = [self.layer1.weight.data,self.layer1.bias.data]
        second = [self.layer2.weight.data,self.layer2.bias.data]
        return first,second
    def forward(self,input): 
        out = self.layer1(input) 
        out = F.relu(out)
        out = self.layer2(out) 
        out = F.softmax(out,dim=1)
        return out

model = MyFirstNetwork(154*154,56*56,2)
model = model.cuda()
print(model)

MyFirstNetwork(
  (layer1): Linear(in_features=23716, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)


### Loss

介紹兩種不同的loss function

In [9]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

分類問題常用的loss function: 交叉熵(cross-entropy)

In [10]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)

### loss(x,class)=−x[class]+log(∑exp(x[j]))

利用上述公式自訂函數與內建函數比較

In [11]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
print(output.data.item())

def CrossEntropyLoss(input,target):
    sum = 0
    for i in range(len(input)):
        tmp = 0
        for j in range(len(input[0])):
            tmp+=np.e**(input[i][j])
        sum+=-input[i][target[i]]+np.log(tmp)
    return float(sum)/len(input)

print(CrossEntropyLoss(input.data.numpy(),target.data.numpy()))

2.2620832920074463
2.262083344402672


### Optimizer

將多個圖形利用自訂的類別建立成一個物件

In [12]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
# ### Optimizer
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(154,154)):
        self.files = glob(root_dir)
        self.size = size
        self.arr = np.array([[0],[1]],int)
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        input = Variable(torch.from_numpy(np.asarray(Image.open(self.files[idx]).resize((self.size)).convert('L'), float).reshape(1,-1)))  
        if(idx<1000):
            target = Variable(torch.from_numpy(self.arr[0]))
        else:
            target = Variable(torch.from_numpy(self.arr[1])) 
        return input,target

dataset = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/classifier2/*.jpg")
test = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/sample_test/*.jpg")

### Training

In [13]:
import torch.optim as optim
from time import perf_counter
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
t1 = perf_counter()
t  = t1
for i in range(100):
    index = 0
    print("epoch",i)
    for input, target in dataset:
        if torch.cuda.is_available():
            input = Variable(input.cuda())
            target = Variable(target.cuda())
        else:
            input, target = Variable(input), Variable(target)    
        output = model(input.float()/(3*255.))
        optimizer.zero_grad()
        loss = loss_fn(output, target.long())
        loss.backward()
        optimizer.step()
        index+=1
        if(perf_counter()-t > 30):
            t = perf_counter()
            print("  Completion ratio :",str(index*100.0/2000)+"%")
    ans=0
    for j in range(10):        
        output = test[j][0].cuda()
        output = model(output.float())     
        tmp = output.data.cpu().numpy()     
        if(tmp[0][0]>tmp[0][1] and j<5):
            ans+=1
        if(tmp[0][0]<tmp[0][1] and j>=5):
            ans+=1
    print("  accuracy :",str(ans*100/10.)+"%")

        
t2  = perf_counter()

print(model)
print("--------------------------------------")
print("layer1 :","\nweight :\n",model.layer()[0][0].cpu().numpy(),"\nbias :\n",model.layer()[0][1].cpu().numpy())
print("")
print("layer2 :","\nweight :\n",model.layer()[1][0].cpu().numpy(),"\nbias :\n",model.layer()[1][1].cpu().numpy())
print("time :",str((t2-t1)/60.0)+"min")
#print("\ntest.....")

epoch 0
  Completion ratio : 8.7%
  Completion ratio : 17.45%
  Completion ratio : 26.15%
  Completion ratio : 34.85%
  Completion ratio : 43.55%
  Completion ratio : 52.25%
  Completion ratio : 60.95%
  Completion ratio : 69.65%
  Completion ratio : 78.35%
  Completion ratio : 87.05%
  Completion ratio : 95.75%
  accuracy : 50.0%
epoch 1
  Completion ratio : 4.45%
  Completion ratio : 13.15%
  Completion ratio : 21.85%
  Completion ratio : 30.55%
  Completion ratio : 39.25%
  Completion ratio : 47.95%
  Completion ratio : 56.65%
  Completion ratio : 65.35%
  Completion ratio : 74.05%
  Completion ratio : 82.75%
  Completion ratio : 91.45%
  accuracy : 50.0%
epoch 2
  Completion ratio : 0.15%
  Completion ratio : 8.85%
  Completion ratio : 17.55%
  Completion ratio : 26.25%
  Completion ratio : 34.95%
  Completion ratio : 43.65%
  Completion ratio : 52.35%
  Completion ratio : 61.05%
  Completion ratio : 69.75%
  Completion ratio : 78.45%
  Completion ratio : 87.15%
  Completion ratio 

KeyboardInterrupt: 